# Writing Exams

- Replying to an email
- Summarizing the contents of the given prompt
- Writing your opinion



In [2]:
import json
from typing import List, Union

from tqdm.notebook import tqdm
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage, StrOutputParser
import pandas as pd

import os
os.environ["OPENAI_API_KEY"] = "Your API Key"

In [3]:
model = ChatOpenAI(model="gpt-3.5-turbo")

## 이메일 답장하기

### 가상의 이메일 생성하기

In [4]:
def build_text_sampling_chain(desc):
    prompt_template = PromptTemplate.from_template(template=desc)
    chain = prompt_template | model | StrOutputParser()
    return chain

In [5]:
email_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 이메일에 답장하기에 등장 할 법한 가상의 영어 이메일 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [6]:
email_gen_chain.invoke({})

'Subject: Re: Job Interview Confirmation\n\nDear Mr. Smith,\n\nThank you for reaching out to me regarding the job interview at XYZ Company. I am writing to confirm my attendance at the scheduled interview on Friday, June 25th at 10:00 AM.\n\nI appreciate the opportunity to interview for the position of Marketing Manager and am looking forward to discussing my qualifications and experience with you and the team.\n\nPlease let me know if there is any additional information or documentation that I should bring to the interview. I am excited about the possibility of joining your team and contributing to the success of XYZ Company.\n\nThank you once again for considering my application.\n\nBest regards,\n\nEmily Johnson'

### 답장 평가하기

In [23]:
def build_eval_chain(instruction, reason_desc, score_desc):
    class Evaluation(BaseModel):
        reason: str = Field(description=reason_desc)
        score: int = Field(description=score_desc)
    
    parser = JsonOutputParser(pydantic_object=Evaluation)
    format_instructions = parser.get_format_instructions()
    
    human_prompt_template = HumanMessagePromptTemplate.from_template(
                                "# Instruction: {instruction}\n"
                                "# Context: {context}\n"
                                "# User: {input}\n"
                                "{format_instructions}")
    
    prompt = ChatPromptTemplate.from_messages([human_prompt_template])
    prompt = prompt.partial(format_instructions=format_instructions, instruction=instruction)
    eval_chain = prompt | model | parser
    return eval_chain

In [24]:
email_eval_chain = build_eval_chain(instruction="User의 응답이 Context의 이메일에 대한 적절한 응답인지 추론하고 평가하라",
                                    reason_desc="User의 응답이 Context의 이메일에 대한 적절한 응답인지에 대한 추론",
                                    score_desc="User의 응답이 Context의 이메일에 대한 적절한 응답인지에 대한 점수, 0~10점")

In [25]:
email = email_gen_chain.invoke({})

In [26]:
email

'Subject: Inquiry about Writing Test Results\n\nDear Mr. Johnson,\n\nI hope this email finds you well. I am writing to inquire about the results of the recent Writing test that I took. I am eager to know how I performed and would appreciate any feedback or suggestions for improvement.\n\nThank you for your time and attention to this matter. I look forward to hearing from you soon.\n\nSincerely,\nEmily Smith'

In [27]:
user_answer = "Dear Emily Thompson, I'm pleased to provide the details of our upcoming event, which will be held on March 15th, 6:00 PM at the Grand Hall, Downtown, and includes a formal dress code, engaging presentations by renowned speakers, and kindly requests your RSVP by March 1st"

In [28]:
eval_result = email_eval_chain.invoke({"context": email, "input": user_answer})

In [14]:
eval_result

{'reason': "The user's response is a suitable and relevant answer to the email inquiry about the upcoming event. It provides the necessary details such as the date, time, location, dress code, guest speakers, and RSVP requirements.",
 'score': 10}

## 제시문 내용 요약하기

### 무작위 글 생성

In [29]:
text_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 단락 요약하기에 등장 할 법한 가상의 영어 단락 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [30]:
text = text_gen_chain.invoke({})
text

"In the bustling city of Rivertown, Sarah and James were preparing for their upcoming wedding. With the help of their friends and family, they were busy making arrangements for the big day. Sarah's dress was a beautiful ivory gown, while James looked sharp in his tailored suit. As the date drew closer, excitement filled the air, and the couple couldn't wait to start their new life together as husband and wife."

### 요약 평가하기

In [31]:
summarization_eval_chain = build_eval_chain(instruction="User의 응답이 Context에 대한 적절한 요약인지 추론하고 평가하라",
                                            reason_desc="User의 응답이 Context에 대한 적절한 요약인지에 대한 추론",
                                            score_desc="User의 응답이 Context에 대한 적절한 요약인지에 대한 점수, 0~10점")

In [32]:
user_answer = "The innovative advancement and increasing adoption of solar power is emerging as a green energy solution for a sustainable future."

In [33]:
summarization_eval_chain.invoke({"context": text,
                                 "input": user_answer})

{'reason': "The user's response is not relevant to the context of a wedding preparation in the bustling city of Rivertown.",
 'score': 7}

## 자신의 의견쓰기

### 무작위 이슈 생성


In [34]:
issue_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 자신의 의견쓰기에 등장 할 법한 무작위 이슈 영어 단락 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [35]:
issue = issue_gen_chain.invoke({})
issue

'Recently, there has been a growing debate surrounding the impact of social media on mental health. While some argue that platforms like Instagram and Twitter provide a sense of community and connection, others believe that they can exacerbate feelings of inadequacy and loneliness. In a study conducted by Dr. Smith from the University of XYZ, it was found that individuals who spend more than two hours a day on social media are more likely to experience symptoms of anxiety and depression. This raises important questions about the role of social media in our lives and how we can strike a healthy balance between virtual interactions and real-life connections.'

### 의견 평가하기

In [36]:
opinion_eval_chain = build_eval_chain(instruction="User의 응답이 Context에 대한 적절한 의견 주장인지 추론하고 평가하라",
                                      reason_desc="User의 응답이 Context에 대한 적절한 의견 주장인지 대한 추론",
                                      score_desc="User의 응답이 Context에 대한 적절한 의견 주장인지에 대한 점수, 0~10점")

In [37]:
user_answer = "Gender-neutral names, while fostering individuality and challenging traditional stereotypes, may also inadvertently obscure cultural heritage and create practical challenges in social interactions."

In [38]:
opinion_eval_chain.invoke({"context": issue,
                           "input": user_answer})

{'reason': "The user's response is a well-reasoned argument that considers both the positive and negative aspects of gender-neutral names, highlighting how they can foster individuality while also potentially obscuring cultural heritage and creating practical challenges in social interactions.",
 'score': 9}